In [2]:
from dotenv import load_dotenv
load_dotenv()
from colorama import Fore, Back, Style

# import FAISS
import pinecone
import os
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()


/Users/spinokiem/miniconda3/envs/inno23/lib/python3.11/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Testing Pinecone

In [3]:
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), 
              environment='gcp-starter' # this thing can be found at your acc at https://app.pinecone.io/
              )

In [4]:
index_name = "itl-knl-base"
print(pinecone.Index(index_name=index_name).describe_index_stats())
docsearch = Pinecone.from_existing_index(index_name, embeddings)

{'dimension': 1536,
 'index_fullness': 0.00286,
 'namespaces': {'': {'vector_count': 286}},
 'total_vector_count': 286}


In [15]:
# query = "What did the president say about Ketanji Brown Jackson"
query="số ngày nghỉ phép có tăng theo năm làm việc hay không?" # chính xác
docs = docsearch.similarity_search_with_score(query=query, k=3)
docs

[(Document(page_content='. 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm 18 ngày làm việc/năm đối với cấp bậc từ Trưởng phòng trở lên (JG 6C đến JG8). Cứ mỗi 05 năm làm việc cho Công ty, số ngày nghỉ phép của Người Lao Động sẽ tăng thêm 01 ngày. Trong trường hợp Người Lao Động làm việc không tròn năm với Công ty, tổng số ngày nghỉ phép năm sẽ được tính theo tỉ lệ từ ngày vào làm. Mỗi tháng người lao động được nghỉ số phép mà Người Lao Động có theo tỉ lệ tháng làm việc, nếu không nghỉ thì ngày phép đó sẽ cộng dồn vào những tháng sau', metadata={'document': 'HR.03.V3.2023. Nội quy Lao động', 'heading 1': 'CHƯƠNG I:  THỜI GIỜ LÀM VIỆC VÀ THỜI GIỜ NGHỈ NGƠI', 'heading 2': 'Điều 3: Nghỉ Phép Hàng Năm', 'heading 3': ''}),
  0.903068542),
 (Document(page_content='. Bất kỳ sự thay đổi nào về số ngày phép đã được chấp thuận trước đó sẽ phải tuân thủ quy trình xin nghỉ phép như lần xin phép ban đầu. Công ty khuyến khích Người Lao Động sử dụng hết n

# Try to link this with GPT

In [11]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

prompt_template = """You are an expert about policies of ITL Corporation, I will ask you a question, and then provide you some chunks of text contain relevant information. 
Try to extract information from the provided text & answer in Vietnamese, prefer bullet-points. If the text does not contain relevant information, you should tell me that you don't have the answer.
Questions: 
{user_question}  

Relevant Information:
{relevant_info}
"""

llm = OpenAI(temperature=0, max_tokens=2048)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)


In [7]:
def qa(user_question):
    # query data from database
    top_k=docsearch.similarity_search_with_score(query=user_question, k=3)
    page_content=[i[0].page_content for i in top_k]
    for i in page_content:
        print(i)
    # relevant_info=[]
    # for index, content in enumerate(top_k): 
    #     chunk=f"textchunk {index+1}:\n{content[0].page_content}"
    #     relevant_info.append(chunk)

    # feed it to GPT using the prompt template
    response = llm_chain(
        {
            'user_question': user_question,
            'relevant_info': page_content#top_k#relevant_info
        }
    )
    print(Fore.RED, "---")
    print(Fore.RED, response['text'])

In [8]:
qa("Nguyễn Quốc Thúc là ai?")

TRẦN TUẤN ANH Đại Diện Theo Pháp Luật và Tổng Giám đốc
NỘI QUY LAO ĐỘNG
. Người Lao Động không được nhận thù lao, quà tặng hoặc các lợi ích phi vật chất khác dưới bất kỳ hình thức nào từ những nhà cung cấp, đối thủ cạnh tranh, khách hàng hoặc từ các cá nhân, đơn vị có quan hệ kinh doanh hoặc đang cung cấp dịch vụ cho Công ty. Trong trường hợp xảy ra xung đột quyền lợi, Tổng giám đốc sẽ xem xét và ra quyết định cuối cùng.
 ---
 
Không có thông tin liên quan.


In [12]:
# qa_assistant("nội quy lao động được soạn thảo bởi ai?") # wrong & so slow ~1min
qa("nội quy lao động của ITL được soạn thảo bởi ai?") # still not getting the right info & so slow ~1min

. Quyết định cách chức sẽ được đưa ra sau một buổi họp giữa Đại Diện Theo Pháp Luật của Người Sử Dụng Lao Động và/hoặc người được Đại Diện Theo Pháp Luật ủy quyền hợp pháp để ký hợp đồng lao động với người lao động, đại diện của tổ chức đại diện tập thể người lao động (nếu không có tổ chức đại diện tập thể người lao động, thì cần có đại diện hoặc ý kiến bằng văn bản về việc xử lý kỷ luật lao động của Liên Đoàn Lao Động quận có liên quan), bất kỳ người làm chứng nào có liên quan và Người Lao Động vi phạm
Nội quy này được công bố cho toàn thể Người Lao Động thuộc Công ty. Những vấn đề chưa được quy định trong Nội quy này hoặc trong pháp luật lao động sẽ được giải quyết bằng thoả thuận giữa Công ty và Người Lao Động trên cơ sở tham khảo các chính sách và quy trình hiện hành của Công ty
. Khiển trách bằng miệng sẽ được đưa ra sau một buổi họp xử lý kỷ luật giữa Đại Diện Theo Pháp Luật của Người Sử Dụng Lao Động và/hoặc người được Đại Diện Theo Pháp Luật ủy quyền hợp pháp để ký hợp đồng lao

In [21]:
qa("số ngày nghỉ có tăng theo thâm niên làm việc không?") # right

. 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm 18 ngày làm việc/năm đối với cấp bậc từ Trưởng phòng trở lên (JG 6C đến JG8). Cứ mỗi 05 năm làm việc cho Công ty, số ngày nghỉ phép của Người Lao Động sẽ tăng thêm 01 ngày. Trong trường hợp Người Lao Động làm việc không tròn năm với Công ty, tổng số ngày nghỉ phép năm sẽ được tính theo tỉ lệ từ ngày vào làm. Mỗi tháng người lao động được nghỉ số phép mà Người Lao Động có theo tỉ lệ tháng làm việc, nếu không nghỉ thì ngày phép đó sẽ cộng dồn vào những tháng sau
. Trở lại làm việc muộn hơn thời gian nghỉ thai sản: Trong trường hợp Người Lao Động nghỉ thai sản muốn nghỉ thêm một thời gian thì phải thông báo và được sự chấp thuận của người quản lý trực tiếp ít nhất trước 10 ngày tính từ ngày kết thúc kỳ nghỉ thai sản của mình. Những ngày nghỉ thêm này được xem như là nghỉ không hưởng lương. Đối với Người Lao Động nam: Người Lao Động nam đang đóng BHXH khi vợ sinh con được nghỉ việc hưởng chế độ 

In [16]:
qa("will annual leave increase with seniority?") # note accurate 29 oct

. Người Lao Động cũng có thể sử dụng một lần phép năm nếu không ảnh hưởng đến công việc. Người Lao Động sau thời gian thử việc mới được sử dụng phép năm (phép năm vẫn được tính từ ngày bắt đầu làm việc). Khi Người Lao Động muốn nghỉ phép phải làm đơn xin phép và được sự đồng ý của cấp quản lý trực tiếp hoặc cấp trên cao hơn theo chính sách nghỉ phép của Công ty. Nếu có bất kỳ thay đổi nào ảnh hưởng đến lịch nghỉ phép, Người Lao Động phải thông báo ngay cho cấp quản lý trực tiếp hoặc cấp trên cao hơn
. 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm 18 ngày làm việc/năm đối với cấp bậc từ Trưởng phòng trở lên (JG 6C đến JG8). Cứ mỗi 05 năm làm việc cho Công ty, số ngày nghỉ phép của Người Lao Động sẽ tăng thêm 01 ngày. Trong trường hợp Người Lao Động làm việc không tròn năm với Công ty, tổng số ngày nghỉ phép năm sẽ được tính theo tỉ lệ từ ngày vào làm. Mỗi tháng người lao động được nghỉ số phép mà Người Lao Động có theo tỉ lệ tháng làm việc

In [8]:
qa("thời gian làm việc của nhân viên ITL?") # right

Thời giờ làm việc 44 giờ/tuần: 08giờ/ngày từ thứ 2 đến thứ 6 và buổi sáng thứ 7. Người Lao Động được nghỉ các ngày thứ 7 trong tháng phụ thuộc vào việc sắp xếp công việc từng Phòng ban, Đội nhóm. Buổi sáng: từ 08 giờ đến 12 giờ. Buổi chiều: từ 13 giờ 30 đến 17 giờ 30. Thời giờ làm việc 48 giờ/tuần: 08 giờ/ngày từ thứ 2 đến thứ 7. Buổi sáng: từ 08 giờ đến 12 giờ. Buổi chiều: từ 13 giờ 30 đến 17 giờ 30
. Buổi chiều: từ 13 giờ 30 đến 17 giờ 30. Thời gian làm việc đặc biệt Do tính chất đặc thù của công việc, thời gian làm việc và thời gian nghỉ ngơi của các Người Lao Động tại hiện trường, Người Lao Động tại kho, Người Lao Động phụ trách giao nhận, bảo vệ, tài xế, tạp vụ… sẽ do Giám đốc/ Trưởng phòng/ Trưởng bộ phận phân công theo nhu cầu hoạt động kinh doanh của Công ty và theo quy định của Bộ luật Lao động
. Tuân thủ quy trình công nghệ, quy trình thực hiện công việc trong tất cả các phần mềm để đảm bảo chất lượng công việc, quy định của phòng ban/Công ty và hoạt động chung của Tập đoàn. 

In [13]:
qa("tell me the working time of ITL employees") # 

. Buổi chiều: từ 13 giờ 30 đến 17 giờ 30. Thời gian làm việc đặc biệt Do tính chất đặc thù của công việc, thời gian làm việc và thời gian nghỉ ngơi của các Người Lao Động tại hiện trường, Người Lao Động tại kho, Người Lao Động phụ trách giao nhận, bảo vệ, tài xế, tạp vụ… sẽ do Giám đốc/ Trưởng phòng/ Trưởng bộ phận phân công theo nhu cầu hoạt động kinh doanh của Công ty và theo quy định của Bộ luật Lao động
Thời giờ làm việc 44 giờ/tuần: 08giờ/ngày từ thứ 2 đến thứ 6 và buổi sáng thứ 7. Người Lao Động được nghỉ các ngày thứ 7 trong tháng phụ thuộc vào việc sắp xếp công việc từng Phòng ban, Đội nhóm. Buổi sáng: từ 08 giờ đến 12 giờ. Buổi chiều: từ 13 giờ 30 đến 17 giờ 30. Thời giờ làm việc 48 giờ/tuần: 08 giờ/ngày từ thứ 2 đến thứ 7. Buổi sáng: từ 08 giờ đến 12 giờ. Buổi chiều: từ 13 giờ 30 đến 17 giờ 30
Làm thêm giờ là khoảng thời gian làm việc ngoài thời giờ làm việc bình thường được quy định tại Điều 1 của Nội quy này. Người Lao Động và cấp quản lý trực tiếp có thể thỏa thuận về việc

In [10]:
# len("""Câu trả lời: 
# - Thời gian làm việc bình thường: 44 giờ/tuần, từ thứ 2 đến thứ 6 và buổi sáng thứ 7. 
# - Buổi sáng: từ 08 giờ đến 12 giờ. 
# - Buổi chiều: từ 13 giờ 30 đến 17 giờ 30. 
# - Thời gian làm việc thêm giờ: thỏa thuận về việc làm việc thêm giờ vào ngày làm việc bình thườ""")

275

In [14]:
qa("một năm tôi có bao nhiêu ngày nghỉ?")

Người Lao Động nghỉ việc riêng hưởng nguyên lương cho trong các trường hợp sau: Kết hôn: nghỉ 03 ngày. Con đẻ, con nuôi kết hôn: nghỉ 01 ngày. Cha đẻ, mẹ đẻ, cha nuôi, mẹ nuôi; cha đẻ, mẹ đẻ, cha nuôi, mẹ nuôi của vợ hoặc chồng; vợ hoặc chồng; con đẻ, con nuôi chết: nghỉ 03 ngày. Ông bà (cả bên Nội và bên Ngoại) chết: nghỉ 01 ngày. Người Lao Động có thể nghỉ không hưởng lương 01 ngày trong các trường hợp sau: Anh, chị, em ruột chết. Bố hoặc mẹ hoặc anh, chị, em ruột kết hôn
. 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm 18 ngày làm việc/năm đối với cấp bậc từ Trưởng phòng trở lên (JG 6C đến JG8). Cứ mỗi 05 năm làm việc cho Công ty, số ngày nghỉ phép của Người Lao Động sẽ tăng thêm 01 ngày. Trong trường hợp Người Lao Động làm việc không tròn năm với Công ty, tổng số ngày nghỉ phép năm sẽ được tính theo tỉ lệ từ ngày vào làm. Mỗi tháng người lao động được nghỉ số phép mà Người Lao Động có theo tỉ lệ tháng làm việc, nếu không nghỉ thì ngày 

In [12]:
len("""Đáp án:
- Người Lao Động có thể nghỉ không hưởng lương 01 ngày trong các trường hợp sau: Anh, chị, em ruột chết. Bố hoặc mẹ hoặc anh, chị, em ruột kết hôn
- 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm
- 18 ngày làm việc/năm đ""")

275

In [23]:
qa("nhân viên từ jg5 trở xuống có bao nhiêu ngày nghỉ 1 năm?")

. 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm 18 ngày làm việc/năm đối với cấp bậc từ Trưởng phòng trở lên (JG 6C đến JG8). Cứ mỗi 05 năm làm việc cho Công ty, số ngày nghỉ phép của Người Lao Động sẽ tăng thêm 01 ngày. Trong trường hợp Người Lao Động làm việc không tròn năm với Công ty, tổng số ngày nghỉ phép năm sẽ được tính theo tỉ lệ từ ngày vào làm. Mỗi tháng người lao động được nghỉ số phép mà Người Lao Động có theo tỉ lệ tháng làm việc, nếu không nghỉ thì ngày phép đó sẽ cộng dồn vào những tháng sau
Người Lao Động được hưởng số ngày phép năm như sau: 12 ngày làm việc/năm đối với người lao động có cấp bậc (JG 4 trở xuống). 14 ngày làm việc/năm đối với người lao động chưa thành niên, lao động là người khuyết tật, người làm nghề, công việc nặng nhọc, độc hại, nguy hiểm. 15 ngày làm việc/năm đối với cấp bậc Giám sát/ Phó phòng (JG5) và tất cả các Người Lao Động thuộc bộ phận hàng hóa hàng không (JG 3 - 5 đối với Bộ phận hàng hóa hàng 

In [24]:
# qa("how many annual leaves do employees JG5 have?") # now after I clean up & structure the data in a better way. it answer this right. 
# qa("how many annual leaves do employees JG4 have?") # JG4 right
qa("how many annual leaves do employees JG3 have?") # JG3 wrong

. 16 ngày làm việc/năm đối với người làm nghề, công việc đặc biệt nặng nhọc, độc hại, nguy hiểm 18 ngày làm việc/năm đối với cấp bậc từ Trưởng phòng trở lên (JG 6C đến JG8). Cứ mỗi 05 năm làm việc cho Công ty, số ngày nghỉ phép của Người Lao Động sẽ tăng thêm 01 ngày. Trong trường hợp Người Lao Động làm việc không tròn năm với Công ty, tổng số ngày nghỉ phép năm sẽ được tính theo tỉ lệ từ ngày vào làm. Mỗi tháng người lao động được nghỉ số phép mà Người Lao Động có theo tỉ lệ tháng làm việc, nếu không nghỉ thì ngày phép đó sẽ cộng dồn vào những tháng sau
. Trường hợp Người Lao Động nữ cấp bậc từ JG4 trở xuống nếu không có nhu cầu nghỉ và được Công ty đồng ý thì Người Lao Động sẽ được chi trả theo quy định của luật và theo chính sách của Công ty.
Người Lao Động được hưởng số ngày phép năm như sau: 12 ngày làm việc/năm đối với người lao động có cấp bậc (JG 4 trở xuống). 14 ngày làm việc/năm đối với người lao động chưa thành niên, lao động là người khuyết tật, người làm nghề, công việc nặ

In [25]:
qa("một năm được nghỉ lễ những ngày nào?")

. Căn cứ vào quy định nghỉ lễ, tình hình hoạt động của Công ty và lịch nghỉ lễ của cơ quan nhà nước hàng năm, Ban Giám đốc sẽ quyết định lịch nghỉ lễ của Công ty và sẽ công bố cho toàn bộ Người Lao Động trước kỳ nghỉ lễ. Đối với Người Lao Động là công dân nước ngoài làm việc tại Công ty, ngoài các ngày nghỉ theo quy định trên, Người Lao Động còn được nghỉ thêm 01 ngày tết cổ truyền dân tộc và 01 ngày Quốc khánh của nước mình.
Tất cả Người Lao Động được nghỉ làm việc và hưởng nguyên lương những ngày lễ sau đây: Tết dương lịch: 01 ngày (ngày 1 tháng 1 dương lịch). Tết Nguyên đán: 05 ngày. Ngày Giỗ Tổ Hùng Vương: 01 ngày (ngày 10 tháng 3 âm lịch). Ngày Chiến thắng: 01 ngày (ngày 30 tháng 4 dương lịch). Ngày Quốc tế lao động: 01 ngày (ngày 1 tháng 5 dương lịch). Ngày Quốc khánh: 02 ngày (ngày 02/9 dương lịch và ngày 01/09 hoặc 03/09 dương lịch). Nếu ngày nghỉ lễ hàng năm đề cập ở trên trùng vào ngày nghỉ cuối tuần thì Người Lao Động được nghỉ bù vào ngày làm việc kế tiếp
Người Lao Động ngh

In [17]:
qa("sinh đôi thì được nghỉ mấy ngày?") # it returns more accurate & more details answer than before

. 10 ngày làm việc nếu sinh đôi, nếu sinh ba trở lên, cứ mỗi con thì được nghỉ thêm 03 ngày làm việc. 14 ngày làm việc nếu sinh đôi trở lên mà phải phẫu thuật. Thời gian nghỉ này được tính trong khoảng 30 ngày đầu kể từ ngày vợ sinh con.
Người Lao Động nghỉ việc riêng hưởng nguyên lương cho trong các trường hợp sau: Kết hôn: nghỉ 03 ngày. Con đẻ, con nuôi kết hôn: nghỉ 01 ngày. Cha đẻ, mẹ đẻ, cha nuôi, mẹ nuôi; cha đẻ, mẹ đẻ, cha nuôi, mẹ nuôi của vợ hoặc chồng; vợ hoặc chồng; con đẻ, con nuôi chết: nghỉ 03 ngày. Ông bà (cả bên Nội và bên Ngoại) chết: nghỉ 01 ngày. Người Lao Động có thể nghỉ không hưởng lương 01 ngày trong các trường hợp sau: Anh, chị, em ruột chết. Bố hoặc mẹ hoặc anh, chị, em ruột kết hôn
Người Lao Động nữ trong thời gian hành kinh được nghỉ mỗi ngày 30 phút, số ngày có thời gian nghỉ trong thời gian hành kinh do hai bên thỏa thuận phù hợp với điều kiện thực tế tại nơi làm việc và nhu cầu của Người Lao Động nữ nhưng tối thiểu là 03 ngày làm việc trong một tháng. Trườ

In [26]:
qa("Những hành vi bị cấm tại nơi làm việc")

. Một số ví dụ về các hành vi bị cấm hoặc bỏ sót bị cấm (nghĩa là, không hành động theo yêu cầu của Nội Quy Lao Động này hoặc theo luật Việt Nam) bao gồm những điều sau: Không tuân thủ các quy trình làm việc và các qui định khác đã được hướng dẫn dẫn đến làm thiệt hại kết quả kinh doanh, hoặc kết quả công việc; Không thực hiện công việc và nhiệm vụ của mình với sự quan tâm và chú ý đúng mức hoặc không đáp ứng các tiêu chuẩn về chất lượng và số lượng công việc
Trong suốt thời gian làm việc và trong phạm vi nơi làm việc, người lao động không được thực hiện các hành vi sau: Hút thuốc trong khu vực quy định không được hút thuốc. Uống rượu, bia, chất kích thích trong giờ làm việc. Bàn tán, cáo buộc sai, cản trở hoạt động vận hành hoặc ngăn cản Người Lao Động khác thực hiện công việc. Có hành vi trái đạo đức hoặc không đứng đắn tại nơi làm việc. Cố ý cưỡng ép, lăng nhục, đe doạ hoặc doạ dẫm Người Lao Động khác. Lăng nhục, đe doạ hoặc doạ dẫm đối tác, khách hàng của Người Sử Dụng Lao Động
. M

In [27]:
qa("trong suốt thời gian làm việc và trong phạm vi nơi làm việc, người lao động không được thực hiện các hành vi nào?")

Trong suốt thời gian làm việc và trong phạm vi nơi làm việc, người lao động không được thực hiện các hành vi sau: Hút thuốc trong khu vực quy định không được hút thuốc. Uống rượu, bia, chất kích thích trong giờ làm việc. Bàn tán, cáo buộc sai, cản trở hoạt động vận hành hoặc ngăn cản Người Lao Động khác thực hiện công việc. Có hành vi trái đạo đức hoặc không đứng đắn tại nơi làm việc. Cố ý cưỡng ép, lăng nhục, đe doạ hoặc doạ dẫm Người Lao Động khác. Lăng nhục, đe doạ hoặc doạ dẫm đối tác, khách hàng của Người Sử Dụng Lao Động
. Trong suốt thời hạn làm việc của mình, Người Lao Động được yêu cầu hành xử theo cách thức phù hợp với nơi làm việc và giữ cho môi trường làm việc không có bạo lực và quấy rối
Người Lao Động làm việc theo thời gian quy định trên vì bất cứ lý do gì mà đến Công ty/ nơi làm việc muộn so với giờ quy định hoặc về trước giờ kết thúc làm việc, được xem là vi phạm nội quy nếu không được sự đồng ý của cấp Quản lý trực tiếp.
 ---
 
Đáp án:
- Không được hút thuốc
- Không đ

In [28]:
qa("thái độ đối với cấp quản lý cao hơn của người lao động phải như thế nào?")

. Nếu thực sự có hoặc có thể có xung đột lợi ích, Người Lao Động nên đưa vấn đề lên người quản lý trực tiếp của người lao động hoặc Người đứng đầu bộ phận Tuân thủ hoặc người đại diện được ủy quyền hợp pháp của Người Sử Dụng Lao Động để giải thích và giải quyết. Trong suốt thời hạn làm việc của mình, Người Lao Động phải đối xử một cách trung thực, công bằng và tôn trọng với Người Sử Dụng Lao Động, người lao động khác, các khách hàng, nhà cung cấp dịch vụ, nhà cung cấp, đối thủ cạnh tranh của Người Sử Dụng Lao Động và những người khác mà Người Sử Dụng Lao Động có quan hệ kinh doanh với họ
. Đối với Người Lao Động mới: Cần được các cấp Quản lý/ Lãnh đạo giới thiệu với các Người Lao Động khác trong ngày nhận việc, quan tâm giúp đỡ, hướng dẫn để họ nắm bắt các quy chế, quy định, chính sách của Công ty và sớm thích nghi với công việc, hòa nhập với môi trường làm việc tại Công ty.
Khi phát hiện Người Lao Động có hành vi vi phạm kỷ luật lao động tại thời điểm xảy ra hành vi

# filter metadata

In [29]:
query="thái độ đối với cấp quản lý cao hơn của người lao động phải như thế nào?"
docsearch.similarity_search_with_score(query=query, k=3) # FAISS is faster

[(Document(page_content='. Nếu thực sự có hoặc có thể có xung đột lợi ích, Người Lao Động nên đưa vấn đề lên người quản lý trực tiếp của người lao động hoặc Người đứng đầu bộ phận Tuân thủ hoặc người đại diện được ủy quyền hợp pháp của Người Sử Dụng Lao Động để giải thích và giải quyết. Trong suốt thời hạn làm việc của mình, Người Lao Động phải đối xử một cách trung thực, công bằng và tôn trọng với Người Sử Dụng Lao Động, người lao động khác, các khách hàng, nhà cung cấp dịch vụ, nhà cung cấp, đối thủ cạnh tranh của Người Sử Dụng Lao Động và những người khác mà Người Sử Dụng Lao Động có quan hệ kinh doanh với họ', metadata={'document': 'HR.03.V3.2023. Nội quy Lao động', 'heading 1': 'CHƯƠNG II:  QUY ĐỊNH VỀ TRẬT TỰ NƠI LÀM VIỆC VÀ ĐẠO ĐỨC KINH DOANH TRONG DOANH NGHIỆP', 'heading 2': 'Điều 9: Ứng xử của Người Lao Động', 'heading 3': ''}),
  0.876327395),
 (Document(page_content='. Đối với Người Lao Động mới: Cần được các cấp Quản lý/ Lãnh đạo giới thiệu với các Người Lao Động kha

In [ ]:
"""
$eq - Equal to (number, string, boolean)
$ne - Not equal to (number, string, boolean)
$gt - Greater than (number)
$gte - Greater than or equal to (number)
$lt - Less than (number)
$lte - Less than or equal to (number)
$in - In array (string or number)
$nin - Not in array (string or number)
"""

In [34]:
docsearch.similarity_search(query=query, filter={
    'heading 2':'Điều 9: Ứng xử của Người Lao Động'
    }
) # this works

[Document(page_content='. Nếu thực sự có hoặc có thể có xung đột lợi ích, Người Lao Động nên đưa vấn đề lên người quản lý trực tiếp của người lao động hoặc Người đứng đầu bộ phận Tuân thủ hoặc người đại diện được ủy quyền hợp pháp của Người Sử Dụng Lao Động để giải thích và giải quyết. Trong suốt thời hạn làm việc của mình, Người Lao Động phải đối xử một cách trung thực, công bằng và tôn trọng với Người Sử Dụng Lao Động, người lao động khác, các khách hàng, nhà cung cấp dịch vụ, nhà cung cấp, đối thủ cạnh tranh của Người Sử Dụng Lao Động và những người khác mà Người Sử Dụng Lao Động có quan hệ kinh doanh với họ', metadata={'document': 'HR.03.V3.2023. Nội quy Lao động', 'heading 1': 'CHƯƠNG II:  QUY ĐỊNH VỀ TRẬT TỰ NƠI LÀM VIỆC VÀ ĐẠO ĐỨC KINH DOANH TRONG DOANH NGHIỆP', 'heading 2': 'Điều 9: Ứng xử của Người Lao Động', 'heading 3': ''}),
 Document(page_content='Trong suốt thời hạn làm việc của mình, Người lao động phải tuân thủ các quy định về phòng chống hối lộ và tham nhũng theo Chính